In [4]:
import re
import opencc


In [17]:
import re

log_entry_pattern = re.compile(
    r"Chinese characters:\s*(?P<chinese>.*?),\s*"
    r"English words:\s*(?P<english>.*?),\s*"
    r"Description:\s*(?P<description>.*?),\s*"
    r"Original:\s*(?P<original>.*?),\s*"
    r"image_name:\s*(?P<image_name>[^\s]+)"
)

# Example log entry

with open('./logs/query_samples_20250326_130651.log', 'r') as f:
    log_entries = f.read()

matches = log_entry_pattern.finditer(log_entries)

for match in matches:
    print(f"Chinese: {match.group('chinese')}")
    print(f"English: {match.group('english')}")
    print(f"Description: {match.group('description')}")
    print(f"Original: {match.group('original')}")
    print(f"Image Name: {match.group('image_name')}")
    print("-" * 40)


Chinese: 
English: centre for nonprofit leadership part of national volunteer  philanthropy centre
Description: triangle curve
Original: 
Image Name: 40201606406U_890d9bd1-9b40-4390-8c2a-4dc832ff3552.jpg
----------------------------------------
Chinese: 
English: compas pass
Description: 
Original: 
Image Name: 40201606407Y_6a234992-d6cc-44f4-8bb3-0d710e86c534.jpg
----------------------------------------
Chinese: 
English: boardmatch board match
Description: 
Original: 
Image Name: 40201606408W_fa1994e2-68d1-4c43-97c6-1157c115bd31.jpg
----------------------------------------
Chinese: 
English: pulsecheck pulse check
Description: 
Original: 
Image Name: 40201606409X_b5a09541-7bc2-46d2-9a4e-79d671b75485.jpg
----------------------------------------
Chinese: 
English: nature green
Description: tree
Original: 
Image Name: 40201606410T_b7c48499-2d9b-448f-80aa-e1a71c3ed651.jpg
----------------------------------------
Chinese: 富國
English: 
Description: 
Original: 富国
Image Name: 40201606416Q_a8

In [14]:
def strip_to_non_repeating(s: str) -> str:
    """Return the smallest non-repeating substring of a repeating string."""
    n = len(s)
    for i in range(1, n // 2 + 1):  # Check possible substring lengths
        if n % i == 0:  # The substring length must evenly divide the full length
            if s[:i] * (n // i) == s:
                return s[:i]
    return s  # Return original string if it's not repeating

In [16]:
with open('./logs/query_samples_20250326_130651.log', 'r') as f:
    msg = f.read()
converter = opencc.OpenCC('t2s.json')
log_entry_pattern = re.compile(
    r"Chinese characters:\s*(?P<chinese>.*?),\s*"
    r"English words:\s*(?P<english>.*?),\s*"
    r"Description:\s*(?P<description>.*?),\s*"
    r"Original:\s*(?P<original>.*?),\s*"
    r"image_name:\s*(?P<image_name>[^\s]+)"
)
# Find all the mismatches in the log content
matches = log_entry_pattern.finditer(msg)
mismatches = []
total = 0
for match in matches:
    chinese = match.group('chinese')
    english = match.group('english')
    description = match.group('description')
    original = match.group('original')
    image_name = match.group('image_name')
    total += 1

    extracted = re.sub(r'[^\u4e00-\u9fff\u3400-\u4DBF\uF900-\uFAFF]', '', chinese)  # Extract both 汉字, CJK Unified Ideographs Extension A, and CJK Compatibility Ideographs
    original = re.sub(r'[^\u4e00-\u9fff\u3400-\u4DBF\uF900-\uFAFF]', '', original)  # Extract both 汉字, CJK Unified Ideographs Extension A, and CJK Compatibility Ideographs
    extracted_simplified = converter.convert(extracted)
    # extracted_simplified = strip_to_non_repeating(extracted_simplified)

    original_simplified = converter.convert(original)
    # original_simplified = strip_to_non_repeating(original_simplified)

    if extracted_simplified != original_simplified:
        print(f"Extracted: {extracted_simplified}, Original: {original_simplified}, File: {image_name}")
        mismatches.append((extracted_simplified, original_simplified, image_name))

    # if original_ != original_simplified:
    #     print("Original:{} Stripped: {} Simplified: {}".format(original_, original, original_simplified))

print(len(mismatches), 1 - len(mismatches) / total)

0 1.0


In [48]:
for extracted, original, file in mismatches:
    print(f"Extracted: {extracted}, Original: {original}, File: {file}")

Extracted: 土俗村蓼鸡汤, Original: 土俗村参鸡汤, File: T1113931Z_T1113931Z.jpg
Extracted: , Original: 嬉戏谷, File: T1113999I_T1113999I.jpg
Extracted: , Original: 泮塘, File: T1114030Z_T1114030Z.jpg
Extracted: 黑蓼秀, Original: 黑参秀, File: 40201400028R_1b25f1ba-8185-465e-bb8d-a71fa09add3d.jpg
Extracted: , Original: 吉, File: 40201400466S_b95659be-d27f-44fb-82a1-5798400148fb.jpg
Extracted: 润楹, Original: 润桜, File: 40201707393W_bb9ed7e4-e90b-4107-8706-9620b8d3d5bc.jpg
Extracted: 皇湾皇家湾, Original: 皇家湾, File: 40201707400Y_ab1b3910-0bd0-48e2-9f65-19f2179eb1bc.jpg
Extracted: 护鱼行动, Original: 护鲨行动, File: 40201518281X_f42b8277-e711-49bd-b5cc-94ae61edfb91.jpg
Extracted: , Original: 苦口良方, File: 40202016565W_ab795331-7bfc-4ed8-9fb0-0daba8c49306.jpeg
Extracted: 黑, Original: 点, File: 40201820557Q_38755a2d-33ce-4ec7-b098-3fc4e1ae78b7.jpg
Extracted: , Original: 珊瑚海日本料理, File: T1404399B_T1404399B.jpg
Extracted: , Original: 出版, File: T1404449B_eac5e537-694c-45e1-80d7-edbcaab5f9d9.BMP
Extracted: 资业保险代理私人有限公司, Original: 宝业保险代理私人

In [39]:
"宝宝" == "宝宝"

True

In [23]:
print(len(mismatches))
TN = len([1 for x, y in matches if x == "None"])
TP = len([1 for x, y in matches if x != "None"])
FN = len([1 for x, y, z in mismatches if y != "None"])
FP = len([1 for x, y, z in mismatches if y == "None"])
print("{} {}\n {} {}".format(TP, FP, FN, TN))
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1 = 2 / (1 / recall + 1 / precision)
acc = (TP + TN) / (TP + TN + FP + FN)
print(" Recall: {:.3f}\n Precision: {:.3f}\n F1 score: {:.3f}\n Accuracy: {:.3f}".format(recall, precision, f1, acc))

267


ValueError: too many values to unpack (expected 2)

In [7]:
for x, y, z in mismatches:
    if y != "None" and x != "None":
        print(x, y)


巴乐 导乐
土俗村蓼鸡汤 土俗村参鸡汤
黑蓼秀 黑参秀
润樱 润桜
皇湾皇家湾 皇家湾
莲昌源 达昌源
护鱼行动 护鲨行动
大乎飞跃 大孚飞跃
黑 点
山 出版
资业保险代理私人有限公司 宝业保险代理私人有限公司
华多 华安
欣艺铭 米欣艺匊
习燕燕窝 习燕燕窝让爱延续
串 宇
浩达食品 沃达食品
麺井 面
焕颜白参磨砂啫喱 焕颜白参磨砂唶喱
明辉活田鸡粥原创者 明𪸩活田鸡粥原创者
桜白 白
桜辉 辉
桜澥 润
悪魔鸡排 恶魔鸡排
翠亨村 翠亨邨
安宜 案萱
艺术 琴韵
音达 韵达
赌场家 娱樂家
红葡萄酒腊肠 红酒腊肠
里红 山里日红
星乃珈琲店 星乃珈啡店
创工 利工
上阁屋 上合屋
沙官 沙宣
奔富 奔冨
雨露 露露
锦 镰仓
日春大地 回春大地
甘露 藏甘露
医武 武医周树生
茉莉亚 茱莉亚
梦雪花秀 韩方雪花秀
理真 宝
光雷子 光电子
一兰始于三十五年前 一兰昭和三十五年创业
新公布 新办公
传远 远传
养生燕窝红利养生轩私人有限公司 养生燕窝红利养生轩
曲巷 自由港
思博资本 思博
未少壹雨 少来未壹
韩国炸鸡 都教授韩国炸鸡
北海道 北雪牛北海道
世纪生化科技的重大发现没有酵素就没有生命 没有酵素就没有生命世纪生化科技的重大发现
海普林 海普森
永井海薙 永永井海苔
南华昌鱼头炉 秋南华昌鱼头炉
亿科私人有限公司 亿科
日本塑料私人有限公司 日本塑胶
维码 三维码
李鹤延年 松鹤延年
狮城磅礴 狮城访谈
酷遇 酪遇
剪刀剪咖哩饭 剪刀剪咖喱饭
君群创 群创
青龙 麦轩
双汇 双辉
金鹭皇家茉莉香米 金鹭皇茉莉香米
金桥鸡午餐肉特级 金桥金桥鸡午餐肉
金桥猪午餐肉特级 金桥金桥猪午餐肉
这非我咖啡 连我咖啡
雪龙 云阴殿
高领 高瓴
宝进财招 招财进宝
有限人公司 新加坡丰成私人有限公司
燕丽人 丽人燕窝
公馆 七公馆
香薰 香熏
瓦引丽 瓦沙丽
橘子快线 橘鲜子快线
欣 欣艺坊
阿粿点姨 阿姨粿点
高明 高盟
化燕阁 华燕阁
王企 宝宝
木鼎雕峰 鼎峰木雕
银座鸭二代目 银座鸭九代目
太阳高 太阳膏
金鑫 鑫
一心 一点一心
御世家晌 御世家扬
桉映 樱咲
南有乔木 南有新田
糖老鸭一只好吃的鸭 糖老鸭 一只好吃的鸭
自家创友诺士肉脞面 友诺士肉脞面自家创
蓝记云吞面家 梦记云吞面家
神楽坂茶寮 神乐坂茶寮
新郎新娘 风华新娘
版面元伊才一 板面
没

In [15]:
import opencc
converter = opencc.OpenCC('t2s.json')
converter.convert('澳寳牌')

'澳寳牌'

In [6]:
with open('logs/extraction_20250324_005903.log', 'r') as f:
    msg = f.read()
mismatch_pattern = r"\[.*?\] ✗ Mismatch - Extracted: '(.*?)', Original: '(.*?)', File: (.*?)\.jpg"

# Find all the mismatches in the log content
matches = re.findall(mismatch_pattern, msg)

# Output the extracted mismatches
for match in matches:
    extracted, original, file = match
    print(f"Extracted: {extracted}, Original: {original}, File: {file}.jpg")

print(len(matches))

Extracted: 土俗村蓼鸡汤, Original: 土俗村参鸡汤, File: T1113931Z_T1113931Z.jpg
Extracted: None, Original: 泮塘, File: T1114030Z_T1114030Z.jpg
Extracted: 黑蓼秀, Original: 黑参秀, File: 40201400028R_1b25f1ba-8185-465e-bb8d-a71fa09add3d.jpg
Extracted: None, Original: 吉, File: 40201400466S_b95659be-d27f-44fb-82a1-5798400148fb.jpg
Extracted: 润梁, Original: 润桜, File: 40201707393W_bb9ed7e4-e90b-4107-8706-9620b8d3d5bc.jpg
Extracted: 皇湾皇家, Original: 皇家湾, File: 40201707400Y_ab1b3910-0bd0-48e2-9f65-19f2179eb1bc.jpg
Extracted: 护鱼行动, Original: 护鲨行动, File: 40201518281X_f42b8277-e711-49bd-b5cc-94ae61edfb91.jpg
Extracted: 锣利鴂, Original: 锣利鸽, File: T1017140F_T1017140F.jpg
Extracted: 大孚飞蛾, Original: 大孚飞跃, File: 40201820555R_4213d5d4-0c7b-43dc-89ae-e00b5d731fce.jpg
Extracted: 资业保险代理私人有限公司, Original: 宝业保险代理私人有限公司, File: T1404879Z_T1404879Z.jpg
Extracted: 习燕燕窝, Original: 习燕燕窝让爱延续, File: 40201511651S_0ff13d57-ee57-4147-b5a1-b02feca8eea2.jpg
Extracted: 欣艺缘, Original: 米欣艺匊, File: 40201511653W_ca230d73-095f-4f48-9590-50fe9f38e8ba

In [4]:
with open('logs/extraction_20250324_025308.log', 'r') as f:
    msg = f.read()

mismatch_pattern = r"\[.*?\] ✗ Mismatch - Extracted: '(.*?)', Original: '(.*?)', File: (.*?)\.jpg"

# Find all the mismatches in the log content
matches = re.findall(mismatch_pattern, msg)

# Output the extracted mismatches
for match in matches:
    extracted, original, file = match
    print(f"Extracted: {extracted}, Original: {original}, File: {file}.jpg")
print(len(matches))

Extracted: None, Original: 嬉戏谷, File: T1113999I_T1113999I.jpg
Extracted: None, Original: 泮塘, File: T1114030Z_T1114030Z.jpg
Extracted: 黑蓼秀, Original: 黑参秀, File: 40201400028R_1b25f1ba-8185-465e-bb8d-a71fa09add3d.jpg
Extracted: None, Original: 吉, File: 40201400466S_b95659be-d27f-44fb-82a1-5798400148fb.jpg
Extracted: 润樱, Original: 润桜, File: 40201707393W_bb9ed7e4-e90b-4107-8706-9620b8d3d5bc.jpg
Extracted: 皇湾皇家, Original: 皇家湾, File: 40201707400Y_ab1b3910-0bd0-48e2-9f65-19f2179eb1bc.jpg
Extracted: 护鱼行动, Original: 护鲨行动, File: 40201518281X_f42b8277-e711-49bd-b5cc-94ae61edfb91.jpg
Extracted: None, Original: 发, File: 40201518282V_b607d438-ffe6-4530-b707-3130694fb7c5.jpg
Extracted: 肌, Original: None, File: 40202016544Y_0b618464-78a7-4a81-8c88-93ffb2304df5.jpg
Extracted: 黑, Original: 点, File: 40201820557Q_38755a2d-33ce-4ec7-b098-3fc4e1ae78b7.jpg
Extracted: 日本料理, Original: 珊瑚海日本料理, File: T1404399B_T1404399B.jpg
Extracted: 资业保险代理私人有限公司, Original: 宝业保险代理私人有限公司, File: T1404879Z_T1404879Z.jpg
Extracted: